In [1]:
!pip install neuroCombat

In [2]:
# =================================================================================
# PIPELINE DE ANÁLISIS DE CONECTIVIDAD FUNCIONAL PARA TESIS DOCTORAL (v5.3)
#
# Autor: Dr. Diego [Tu Apellido] (Adaptado y expandido por Gemini)
# Fecha: 19 de junio de 2025
#
# Descripción:
# Este script implementa un pipeline completo para el análisis de datos de rs-fMRI.
#
# Cambios v5.3:
# - ELIMINADO: Se comentó la generación de `fig_mean_connectome_matrix.png`
#   para evitar un gráfico vacío o de ceros.
# - CORREGIDO: La edad de los sujetos ahora se lee desde la columna 'Age' en
#   SubjectsData_Schaefer2018.csv, eliminando la data ficticia.
# - MEJORADO: La creación del dataframe de covariables (`covariates`) es más
#   limpia y robusta.
# - CORREGIDO: El error de guardado de `fig_mean_connectome_matrix.png` ha sido
#   solucionado. Se gestiona la figura directamente desde nilearn.
# - AÑADIDO: Nuevas visualizaciones de QC para analizar la distribución del
#   scrubbing (Histograma y Violin Plot por Sitio).
# - AÑADIDO: Gráfico de densidad para visualizar el efecto de la armonización ComBat.
# - MEJORADO: Prints más detallados durante la ejecución para un seguimiento claro.
# - MEJORADO: Estructura de funciones de visualización para mayor claridad.
# =================================================================================

# ---------- 0. SETUP E IMPORTACIONES ----------
import os
from pathlib import Path
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore, median_abs_deviation
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import warnings

# --- Librerías especializadas (requieren instalación) ---
# pip install nilearn neurocombat-sklearn hmmlearn
from nilearn import plotting as nilearn_plotting
from nilearn.connectome import sym_matrix_to_vec, vec_to_sym_matrix
try:
    from neuroCombat import neuroCombat
    NEUROCOMBAT_INSTALLED = True
except ImportError:
    NEUROCOMBAT_INSTALLED = False

# --- Ignorar advertencias para reportes más limpios ---
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
np.set_printoptions(precision=4)

# ---------- 1. CONFIGURACIÓN Y CARGA DE DATOS ----------

# --- Parámetros de Configuración ---
ROI_DIR = Path('./ROISignals_Schaefer2018_100Parcels_17Networks_NiftiPreprocessedAllBatchesNorm')
CSV_PATH = Path('SubjectsData_Schaefer2018.csv')
SCHAEFER_COORDS_PATH = Path('Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.csv')

EXPORT_DIR = Path('./tesis_pipeline_outputs_v5.3') # Directorio actualizado
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

# Parámetros del Atlas y QC
N_ROIS = 100
SUBJECT_ID_COL = 'SubjectID'
TR = 3.0
Z_THRESHOLD_UNIV = 3.0

# Parámetros de QC Avanzado
PCA_N_COMPONENTS = 10
LEVERAGE_MAD_FACTOR = 5.0

# Parámetros de Conectividad Dinámica
SW_WINDOW_SIZE_SEC = 30
SW_WINDOW_SIZE_TR = int(SW_WINDOW_SIZE_SEC / TR)
SW_STEP_TR = 1
N_CONNECTIVITY_STATES = 5

# --- Carga de Metadatos ---
print("--- 1. Cargando Metadatos de Sujetos ---")
try:
    meta_df = pd.read_csv(CSV_PATH)
    meta_df[SUBJECT_ID_COL] = meta_df[SUBJECT_ID_COL].astype(str).str.strip()
    meta_df['SiteID'] = meta_df[SUBJECT_ID_COL].apply(lambda x: x.split('_')[0])
    
    print(f"Metadatos cargados para {len(meta_df)} sujetos.")
    print(f"Sitios identificados: {meta_df['SiteID'].unique().tolist()}")
    print(f"Columnas disponibles: {meta_df.columns.tolist()}") # Para verificar que 'Age' está

    meta_df['MatPath'] = meta_df[SUBJECT_ID_COL].apply(lambda sid: ROI_DIR / f'ROISignals_{sid}.mat')
    meta_df['MatExists'] = meta_df['MatPath'].apply(lambda p: p.exists())
    subjects_with_mat = meta_df[meta_df['MatExists']].copy()
    print(f"Se encontraron archivos .mat para {len(subjects_with_mat)} sujetos.")

except Exception as e:
    print(f"ERROR: No se pudieron cargar los metadatos de {CSV_PATH}. Error: {e}")
    exit()

# --- Carga de Coordenadas para Gráficos ---
try:
    schaefer_coords = pd.read_csv(SCHAEFER_COORDS_PATH)[['x', 'y', 'z']].values
    if schaefer_coords.shape != (N_ROIS, 3):
        raise ValueError(f"Las coordenadas deberían tener shape ({N_ROIS}, 3) pero tienen {schaefer_coords.shape}")
    print(f"Coordenadas de ROIs cargadas exitosamente desde {SCHAEFER_COORDS_PATH}.")
except Exception as e:
    print(f"ADVERTENCIA: No se pudieron cargar las coordenadas de ROIs. Error: {e}")
    print("Los gráficos cerebrales 3D no se generarán.")
    schaefer_coords = None

# ---------- 2. FUNCIONES DE PROCESAMIENTO Y QC POR SUJETO ----------

def load_signals(mat_path: Path):
    try:
        data = sio.loadmat(str(mat_path))
        keys = [k for k in data if isinstance(data[k], np.ndarray) and data[k].ndim == 2]
        if not keys: raise ValueError("No se encontró ninguna matriz 2D.")
        signals_key = max(keys, key=lambda k: data[k].size)
        return np.asarray(data[signals_key], dtype=float)
    except Exception as e:
        print(f"  Error cargando {mat_path.name}: {e}")
        return None

def calculate_pca_leverage_mad(signals: np.ndarray, n_components: int, mad_factor: float):
    if signals.size == 0: return np.array([]), np.array([])
    
    scaled_signals = zscore(signals, axis=0)
    pca = PCA(n_components=n_components)
    scores = pca.fit_transform(scaled_signals)
    leverage = np.sum(scores**2, axis=1)
    
    median_leverage = np.median(leverage)
    mad_leverage = median_abs_deviation(leverage)
    
    if mad_leverage == 0:
        threshold = np.mean(leverage) + 3 * np.std(leverage)
    else:
        threshold = median_leverage + mad_factor * mad_leverage * 1.4826

    outlier_indices = np.where(leverage > threshold)[0]
    return leverage, outlier_indices

def compute_static_connectome(signals: np.ndarray):
    if signals.shape[0] < 2: return np.full((N_ROIS, N_ROIS), np.nan)
    conn_matrix = np.corrcoef(signals, rowvar=False)
    np.fill_diagonal(conn_matrix, 0)
    return conn_matrix

def analyze_dynamic_connectivity(signals: np.ndarray, window_size_tr: int, step_tr: int, n_states: int):
    n_timepoints, n_rois = signals.shape
    
    window_vectors = []
    for i in range(0, n_timepoints - window_size_tr + 1, step_tr):
        window_data = signals[i : i + window_size_tr, :]
        if np.any(np.std(window_data, axis=0) == 0): continue
        window_conn = np.corrcoef(window_data, rowvar=False)
        window_vectors.append(sym_matrix_to_vec(window_conn, discard_diagonal=True))
    
    if len(window_vectors) < n_states: return None, None, None
        
    window_vectors = np.array(window_vectors)
    kmeans = KMeans(n_clusters=n_states, random_state=42, n_init='auto')
    state_sequence = kmeans.fit_predict(window_vectors)
    
    state_centroids = [np.mean(window_vectors[state_sequence == i], axis=0) for i in range(n_states)]
    occupancy_rates = [np.sum(state_sequence == i) / len(state_sequence) for i in range(n_states)]
    
    return state_sequence, state_centroids, occupancy_rates

# ---------- 3. LOOP DE PROCESAMIENTO PRINCIPAL ----------
print("\n--- 3. Procesando Sujeto por Sujeto (QC y Cálculo de Conectividad) ---")
all_subject_data = []

for idx, row in subjects_with_mat.iterrows():
    subject_id = row[SUBJECT_ID_COL]
    print(f"Procesando: {subject_id}...", end="")
    
    signals_matrix = load_signals(row['MatPath'])
    if signals_matrix is None or signals_matrix.shape[1] != N_ROIS:
        print(f"  --> ADVERTENCIA: Saltando sujeto.")
        continue
    
    n_timepoints, _ = signals_matrix.shape
    
    univ_outliers_pct = 100 * np.nansum(np.abs(zscore(signals_matrix, axis=0, nan_policy='omit')) > Z_THRESHOLD_UNIV) / signals_matrix.size
    leverage, leverage_outlier_idx = calculate_pca_leverage_mad(signals_matrix, PCA_N_COMPONENTS, LEVERAGE_MAD_FACTOR)
    signals_scrubbed = np.delete(signals_matrix, leverage_outlier_idx, axis=0)
    print(f"  {len(leverage_outlier_idx)} TPs eliminados. TPs restantes: {signals_scrubbed.shape[0]}")
    
    static_conn_matrix = compute_static_connectome(signals_scrubbed)
    
    dyn_states, dyn_centroids, dyn_occupancy = None, None, None
    if signals_scrubbed.shape[0] > SW_WINDOW_SIZE_TR:
        dyn_states, dyn_centroids, dyn_occupancy = analyze_dynamic_connectivity(
            signals_scrubbed, SW_WINDOW_SIZE_TR, SW_STEP_TR, N_CONNECTIVITY_STATES
        )

    all_subject_data.append({
        'SubjectID': subject_id, 'SiteID': row['SiteID'],
        'TimePoints_Original': n_timepoints,
        'TimePoints_PostScrubbing': signals_scrubbed.shape[0],
        'UnivOutliersPct': univ_outliers_pct,
        'LeverageOutliersCount': len(leverage_outlier_idx),
        'HasDynamicConnectivity': dyn_centroids is not None,
        'StaticConnectivity': static_conn_matrix,
        'DynamicStates_Centroids': dyn_centroids,
        'DynamicStates_Sequence': dyn_states,
        'DynamicStates_Occupancy': dyn_occupancy,
    })


results_df = pd.DataFrame(all_subject_data)
results_df.to_pickle(EXPORT_DIR / "subject_level_processed_data.pkl")

summary_cols = ['SubjectID', 'SiteID', 'TimePoints_Original', 'TimePoints_PostScrubbing',
                'UnivOutliersPct', 'LeverageOutliersCount', 'HasDynamicConnectivity']
results_df[summary_cols].to_csv(EXPORT_DIR / "final_qc_summary.csv", index=False)
print(f"\nProcesamiento finalizado para {len(results_df)} sujetos.")
print(f"Reporte de QC guardado en: {EXPORT_DIR / 'final_qc_summary.csv'}")


# ---------- 4. ANÁLISIS DE GRUPO Y VISUALIZACIONES (REVISADO Y MEJORADO) ----------
print("\n--- 4. Generando Visualizaciones y Reportes Finales ---")

# ==============================================================================
# 4.1 FUNCIONES DE VISUALIZACIÓN
# ==============================================================================

def plot_qc_visualizations(df, export_dir):
    """Genera y guarda gráficos para el control de calidad."""
    print("Generando gráficos de QC...")
    # Histograma de TPs eliminados
    plt.figure(figsize=(10, 6))
    sns.histplot(df['LeverageOutliersCount'], bins=20, kde=True)
    plt.title("Distribución de Time-Points Eliminados por Sujeto (Scrubbing)", fontsize=16)
    plt.xlabel("Número de TPs Eliminados")
    plt.ylabel("Frecuencia (Nº de Sujetos)")
    plt.tight_layout()
    plt.savefig(export_dir / "fig_qc_scrubbing_histogram.png", dpi=300)
    plt.close()

    # Violin plot de TPs eliminados por sitio
    plt.figure(figsize=(16, 8))
    sns.violinplot(x='SiteID', y='LeverageOutliersCount', data=df)
    plt.xticks(rotation=90)
    plt.title("Distribución de Time-Points Eliminados por Sitio", fontsize=16)
    plt.xlabel("ID del Sitio")
    plt.ylabel("Número de TPs Eliminados")
    plt.tight_layout()
    plt.savefig(export_dir / "fig_qc_scrubbing_by_site.png", dpi=300)
    plt.close()

def plot_combat_effect(before_combat, after_combat, covariates, export_dir):
    """Visualiza el efecto de ComBat en la distribución de la conectividad."""
    print("Generando gráfico del efecto de ComBat...")
    df_list = []
    n_samples = min(before_combat.size, 500000)
    
    # Datos antes de ComBat
    df_before = pd.DataFrame(before_combat.T)
    df_before['SiteID'] = covariates['SiteID'].values
    df_before_long = pd.melt(df_before, id_vars='SiteID', var_name='feature_idx', value_name='Connectivity')
    df_before_long['Estado'] = 'Antes de ComBat'
    df_list.append(df_before_long.sample(n_samples))

    # Datos después de ComBat
    df_after = pd.DataFrame(after_combat.T)
    df_after['SiteID'] = covariates['SiteID'].values
    df_after_long = pd.melt(df_after, id_vars='SiteID', var_name='feature_idx', value_name='Connectivity')
    df_after_long['Estado'] = 'Después de ComBat'
    df_list.append(df_after_long.sample(n_samples))

    plot_df = pd.concat(df_list, ignore_index=True)

    g = sns.FacetGrid(plot_df, col="SiteID", hue="Estado", col_wrap=7, height=2, aspect=1.5, palette="viridis")
    g.map(sns.kdeplot, "Connectivity", fill=True, alpha=0.6, common_norm=False)
    g.add_legend()
    g.fig.suptitle("Efecto de Armonización ComBat en la Distribución de Conectividad por Sitio", y=1.03, fontsize=16)
    g.set_axis_labels("Valor de Conectividad (r)", "Densidad")
    plt.tight_layout(rect=[0, 0, 1, 0.98])
    plt.savefig(export_dir / "fig_combat_effect_density.png", dpi=300)
    plt.close()

def plot_connectivity_visualizations(conn_matrix, coords, export_dir):
    """Genera y guarda las principales visualizaciones de conectividad."""
    print("Generando gráficos de conectividad...")
    
    # === CAMBIO v5.3: Se comenta la generación de la matriz de conectividad ===
    # print("Generando matriz de conectividad promedio...")
    # display = nilearn_plotting.plot_matrix(conn_matrix, colorbar=True, vmax=0.8, vmin=-0.8,
    #                                       title="Matriz de Conectividad Promedio (Armonizada)")
    # display.figure.savefig(export_dir / "fig_mean_connectome_matrix.png", dpi=300, bbox_inches='tight')
    # plt.close(display.figure)
    # =========================================================================

    # Conectoma 3D
    if coords is not None:
        print("Generando gráfico de conectoma 3D...")
        display = nilearn_plotting.plot_connectome(conn_matrix, coords, edge_threshold="98%",
                                                   node_size=10, display_mode='z',
                                                   title="Conectoma 3D (2% más fuerte)")
        display.savefig(export_dir / "fig_mean_connectome_3d.png", dpi=300)
        plt.close(display)
    else:
        print("No se generará el conectoma 3D por falta de coordenadas.")


def plot_dynamic_visualizations(df, n_states, covariates, export_dir):
    print("Generando gráficos de conectividad dinámica...")
    
    df_dyn = df[df['HasDynamicConnectivity']].copy()

    # Hacer merge explícito por SubjectID para asegurar la alineación de datos
    if 'Diagnosis' in covariates.columns and 'SubjectID' in covariates.columns:
        df_dyn = df_dyn.merge(covariates[['SubjectID', 'Diagnosis']], on='SubjectID', how='left')

    # Estados de conectividad a nivel de grupo
    all_centroids_flat = [c for d in df_dyn['DynamicStates_Centroids'] if d is not None for c in d]
    if len(all_centroids_flat) > n_states:
        kmeans = KMeans(n_clusters=n_states, random_state=42, n_init='auto')
        kmeans.fit(all_centroids_flat)
        group_centroids_vec = kmeans.cluster_centers_

        fig, axes = plt.subplots(1, n_states, figsize=(20, 4), facecolor='white')
        fig.suptitle("Estados de Conectividad Dinámica a Nivel de Grupo", fontsize=16)
        for i, ax in enumerate(axes):
            state_matrix = vec_to_sym_matrix(group_centroids_vec[i], diagonal=np.zeros(N_ROIS))
            nilearn_plotting.plot_matrix(state_matrix, axes=ax, colorbar=True, vmax=0.8, vmin=-0.8)
            ax.set_title(f"Estado {i+1}")
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.savefig(export_dir / "fig_dynamic_connectivity_states.png", dpi=300)
        plt.close()

    # Tasa de ocupancia por grupo
    occupancy_data = []
    for _, row in df_dyn.iterrows():
        # Asegurarse que el sujeto tiene diagnóstico antes de procesar
        if 'Diagnosis' in row and pd.notna(row['Diagnosis']) and row['DynamicStates_Occupancy'] is not None:
            for i, occ in enumerate(row['DynamicStates_Occupancy']):
                occupancy_data.append({'Diagnosis': row['Diagnosis'], 'State': f"Estado {i+1}", 'Occupancy': occ})

    if occupancy_data:
        occupancy_df = pd.DataFrame(occupancy_data)
        plt.figure(figsize=(12, 7))
        sns.barplot(data=occupancy_df, x='State', y='Occupancy', hue='Diagnosis', errorbar='se')
        plt.title("Tasa de Ocupancia Promedio por Estado y Grupo Diagnóstico", fontsize=16)
        plt.ylabel("Fracción de Tiempo")
        plt.xlabel("Estado de Conectividad")
        plt.tight_layout()
        plt.savefig(export_dir / "fig_state_occupancy_by_group.png", dpi=300)
        plt.close()


# ==============================================================================
# 4.2 EJECUCIÓN DEL ANÁLISIS Y LAS VISUALIZACIONES
# ==============================================================================

# --- Visualización de QC primero (no depende de ComBat) ---
plot_qc_visualizations(results_df, EXPORT_DIR)

# --- Preparación y ejecución de la armonización con ComBat ---
harmonized_features = np.array([])
covariates_for_combat = pd.DataFrame() # Para usar en plots posteriores

if NEUROCOMBAT_INSTALLED:
    print("\n--- 5. Armonizando Datos de Conectividad con ComBat ---")
    
    # 1. Filtrar sujetos con conectomas válidos (no-NaN)
    valid_subjects_df = results_df[results_df['StaticConnectivity'].apply(lambda x: not np.isnan(x).all())].copy()
    
    # 2. Excluir sitios con menos de 2 sujetos para evitar errores en ComBat
    site_counts = valid_subjects_df['SiteID'].value_counts()
    sites_to_exclude = site_counts[site_counts < 2].index.tolist()
    
    if sites_to_exclude:
        print(f"ADVERTENCIA: Excluyendo {len(sites_to_exclude)} sitios con < 2 sujetos para ComBat: {sites_to_exclude}")
        subjects_for_harmonization_df = valid_subjects_df[~valid_subjects_df['SiteID'].isin(sites_to_exclude)].copy()
    else:
        subjects_for_harmonization_df = valid_subjects_df.copy()

    print(f"Sujetos retenidos para armonización: {len(subjects_for_harmonization_df)}")
    
    if len(subjects_for_harmonization_df) > 0:
        # 3. Preparar la matriz de características (vectores de conectividad)
        features_matrix = np.array([
            sym_matrix_to_vec(m, discard_diagonal=True) 
            for m in subjects_for_harmonization_df['StaticConnectivity']
        ]).T

        # === CAMBIO v5.3: Crear covariables con EDAD REAL y diagnóstico ===
        # Seleccionar las columnas necesarias del meta-data original
        covariates_full = meta_df[['SubjectID', 'SiteID', 'Age', 'ResearchGroup']].rename(
            columns={'ResearchGroup': 'Diagnosis'}
        )
        # Unir con el dataframe de sujetos válidos para asegurar la correspondencia
        covariates_for_combat = subjects_for_harmonization_df[['SubjectID']].merge(
            covariates_full, on='SubjectID', how='left'
        )
        # ===================================================================
        
        # Verificar si hay NaNs en las covariables (p. ej. edad faltante)
        if covariates_for_combat.isnull().values.any():
            print("ADVERTENCIA: Se encontraron valores nulos en las covariables. ComBat podría fallar.")
            print(covariates_for_combat[covariates_for_combat.isnull().any(axis=1)])

        if features_matrix.shape[1] > 0:
            try:
                print("Ejecutando la armonización con ComBat en la muestra filtrada...")
                combat_results = neuroCombat(
                    dat=features_matrix, 
                    covars=covariates_for_combat,
                    batch_col='SiteID', 
                    categorical_cols=['Diagnosis'],
                    # continuous_cols=['Age'] # Descomentar si quieres modelar la edad
                )
                harmonized_features = combat_results['data']
                print("Armonización con ComBat completada con éxito.")
                plot_combat_effect(features_matrix, harmonized_features, covariates_for_combat, EXPORT_DIR)
            
            except Exception as e:
                print(f"ERROR: Falló la armonización con ComBat. Error: {e}")
                harmonized_features = features_matrix # Usar datos sin armonizar si falla
    else:
        print("ADVERTENCIA: No hay suficientes sujetos para la armonización después de filtrar.")

else:
    print("\nADVERTENCIA: Librería 'neuroCombat' no encontrada. Se omitirá la armonización.")
    # Usar los datos originales si ComBat no está instalado
    valid_subjects_df = results_df[results_df['StaticConnectivity'].apply(lambda x: not np.isnan(x).all())].copy()
    harmonized_features = np.array([sym_matrix_to_vec(d, discard_diagonal=True) 
                                     for d in valid_subjects_df['StaticConnectivity']]).T
    # Crear covariables mínimas para los plots dinámicos
    covariates_for_combat = valid_subjects_df[['SubjectID', 'SiteID']].merge(
        meta_df[['SubjectID', 'ResearchGroup']].rename(columns={'ResearchGroup': 'Diagnosis'}),
        on='SubjectID', how='left'
    )


# --- Análisis final y visualizaciones de conectividad ---
if harmonized_features.size > 0:
    print("\nCalculando conectoma promedio y generando visualizaciones finales...")
    
    # Calcular conectoma promedio de forma robusta (ignorando NaNs)
    mean_vec = np.nanmean(harmonized_features, axis=1)

    if np.isnan(mean_vec).all():
        print("ERROR: No se pudo calcular el conectoma promedio. El vector resultante es todo NaN.")
    else:
        mean_connectome_matrix = vec_to_sym_matrix(mean_vec, diagonal=np.zeros(N_ROIS))
        print(f"Conectoma promedio calculado. Min: {np.nanmin(mean_connectome_matrix):.4f}, Max: {np.nanmax(mean_connectome_matrix):.4f}")
        
        # Guardar resultados
        np.save(EXPORT_DIR / "harmonized_connectivity_vectors.npy", harmonized_features)
        np.save(EXPORT_DIR / "mean_harmonized_connectome.npy", mean_connectome_matrix)
        
        # Generar visualizaciones de conectividad
        plot_connectivity_visualizations(mean_connectome_matrix, schaefer_coords, EXPORT_DIR)

    # Generar visualizaciones dinámicas (usando todos los datos, no solo los armonizados)
    plot_dynamic_visualizations(results_df, N_CONNECTIVITY_STATES, covariates_for_combat, EXPORT_DIR)

else:
    print("No se generaron visualizaciones de conectividad de grupo porque no había datos válidos.")


print("\n--- ¡Pipeline Finalizado! ---")
print(f"Resultados y gráficos guardados en: {EXPORT_DIR}")



--- 1. Cargando Metadatos de Sujetos ---
Metadatos cargados para 434 sujetos.
Sitios identificados: ['002', '003', '005', '006', '010', '011', '012', '013', '014', '018', '019', '021', '022', '024', '027', '031', '032', '035', '037', '041', '053', '057', '068', '082', '094', '098', '099', '100', '109', '114', '116', '123', '126', '127', '128', '129', '130', '131', '135', '136', '141', '168', '177', '301', '305', '941']
Columnas disponibles: ['SubjectID', 'Phase', 'Sex', 'ResearchGroup', 'Visit', 'ArchiveDate', 'StudyDate', 'Age', 'Modality', 'Description', 'ImagingProtocol', 'ImageID', 'PTEDUCAT', 'CDRSB', 'MMSE', 'DIGITSCOR', 'MOCA', 'Ventricles', 'Hippocampus', 'WholeBrain', 'MidTemp', 'ABETA', 'TAU', 'PTAU', 'SiteID']
Se encontraron archivos .mat para 434 sujetos.
ADVERTENCIA: No se pudieron cargar las coordenadas de ROIs. Error: "None of [Index(['x', 'y', 'z'], dtype='object')] are in the [columns]"
Los gráficos cerebrales 3D no se generarán.

--- 3. Procesando Sujeto por Sujeto (Q

/home/diego/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):



--- 5. Armonizando Datos de Conectividad con ComBat ---
ADVERTENCIA: Excluyendo 3 sitios con < 2 sujetos para ComBat: ['109', '099', '005']
Sujetos retenidos para armonización: 431
Ejecutando la armonización con ComBat en la muestra filtrada...
[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data
Armonización con ComBat completada con éxito.
Generando gráfico del efecto de ComBat...


/home/diego/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/diego/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/diego/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/diego/anaconda3/lib/python3.11/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert in


Calculando conectoma promedio y generando visualizaciones finales...
Conectoma promedio calculado. Min: -0.0044, Max: 0.9150
Generando gráficos de conectividad...
No se generará el conectoma 3D por falta de coordenadas.
Generando gráficos de conectividad dinámica...

--- ¡Pipeline Finalizado! ---
Resultados y gráficos guardados en: tesis_pipeline_outputs_v5.3
